In [4]:
import os
import shutil
import tensorflow as tf

from azureml.core import Environment, VERSION, Workspace, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, StepSequence

print(f"Azure ML SDK Version: {VERSION}")
print(f"Tensorflow Version: {tf.version.VERSION}")

Azure ML SDK Version: 1.42.0
Tensorflow Version: 2.5.3


### Get the actual ML Workspace

In [5]:
DATASET_NAME = "UNIMIB2016-label_20220827_100542"
COMPUTE_NAME = "ML-Workspa-com1"
PIPELINE_NAME = "food-segmentation"

In [6]:
azure_ml_workspace = Workspace.from_config()

### Create experiment

In [7]:
mask_rcnn_experiment = Experiment(workspace=azure_ml_workspace, name="food-segmentation")

### Create environment

In [8]:
pipeline_env = Environment.from_dockerfile("food-segmentation", 
                                            dockerfile="../pipeline/Dockerfile",
                                            pip_requirements="../pipeline/requirements.txt")

pipeline_env.register(workspace=azure_ml_workspace)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM tensorflow/tensorflow:2.5.0-gpu\n\n# Fix for the public key error\n# https://github.com/NVIDIA/nvidia-docker/issues/1632\nRUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub\nRUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/7fa2af80.pub\n\n\n# Install Common Dependencies\nRUN apt-get update && \\\n    apt-get install -y --no-install-recommends \\\n    wget \\\n    ffmpeg \\\n    libsm6 \\\n    libxext6\n\n# Conda Environment\nENV MINICONDA_VERSION py38_4.11.0\nENV PATH /opt/miniconda/bin:$PATH\nRUN wget -qO /tmp/miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-${MINICONDA_VERSION}-Linux-x86_64.sh && \\\n

### Get the compute cluster

In [9]:
compute_target_cc = azure_ml_workspace.compute_targets[COMPUTE_NAME]

### Create run configuration

In [10]:
train_run_config = RunConfiguration()
train_run_config.environment = Environment.get(workspace=azure_ml_workspace, name=pipeline_env.name)

### Create train step

In [11]:
download_folder = "download"
train_arguments = [
    "--dataset_name", DATASET_NAME,
    "--download_folder", download_folder,
    "--epochs", 50
    ]

In [12]:
train_script_step = PythonScriptStep(
    name="Train Mask R-CNN",
    source_directory="./..",
    script_name="train.py",
    arguments=train_arguments,
    compute_target=compute_target_cc,
    allow_reuse=False,
    runconfig=train_run_config
)

### Create pipeline

In [13]:
step_sequence = StepSequence(steps=[
    train_script_step
])

mask_rcnn_pipeline = Pipeline(
    workspace=azure_ml_workspace,
    steps=[step_sequence]
)

### Publish pipeline

In [14]:
published_pipeline = mask_rcnn_pipeline.publish(PIPELINE_NAME, version="0.0.1")

Created step Train Mask R-CNN [4e15873f][679fb522-87c5-42ce-96b1-3b5859152462], (This step will run and generate new outputs)
